In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import libraries
import numpy as np
import sys
#import psi4
np.set_printoptions(threshold=sys.maxsize)
#psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt
import json
import os

# import libraries
import numpy as np
import sys
#import psi4
np.set_printoptions(threshold=sys.maxsize)
#psi4.core.set_output_file('output.dat', False)
import time
import json
import matplotlib.pyplot as plt
import json
import os 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


$$ |\psi_n^{(k)}\rangle = \sum_{m \neq n} \frac{\langle \psi_m^{(0)} | \hat{H}' | \psi_n^{(k-1)} \rangle}{E_n^{(0)} - E_m^{(0)}} |\psi_m^{(0)} \rangle $$
$$ E_n^{(k)} = \langle \psi_n^{(0)} | \hat{H}' | \psi_n^{(k-1)} \rangle $$

Note:

$$ \hat{U}_{CS} \hat{b} \hat{U}^{\dagger}_{CS} = \hat{b} - z $$

$$ \hat{U}_{CS} \hat{b}^{\dagger} \hat{U}^{\dagger}_{CS} = \hat{b}^{\dagger} - z $$

where 

$$ z = -\frac{\langle \hat{d} \rangle}{\sqrt{2 \omega}} $$


In [52]:



class PT_arbitrary:

    def __init__(self, energies, dipoles):
        self.E_array = energies
        self.dipoles = dipoles




    def create_annihilation_operator(self,N):
        """
        Creates the matrix representation of the annihilation operator (b) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the annihilation operator.
        """
        b = np.zeros((N, N))
        
        for j in range(1, N):
            b[j-1, j] = np.sqrt(j)
        
        return b

    def create_creation_operator(self,N):
        """
        Creates the matrix representation of the creation operator (b†) for a harmonic oscillator
        in a Hilbert space with N levels.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the creation operator.
        """
        b_dagger = np.zeros((N, N))
        
        for j in range(1, N):
            b_dagger[j, j-1] = np.sqrt(j)
        
        return b_dagger



    def create_number_operator(self,N):
        """
        Creates the matrix representation of the number operator (n) = b† * b.
        
        Parameters:
        N (int): Number of levels.
        
        Returns:
        np.ndarray: The matrix representation of the number operator.
        """
        b = self.create_annihilation_operator(N)
        b_dagger = self.create_creation_operator(N)
        
        # The number operator is n = b† * b
        n = np.dot(b_dagger, b)
        
        return n



    def build_d_array(
        self,
        n_el,
        lambda_vector,
        mu_array,
        coherent_state=False,
        coherent_state_pos = None,
        coherent_state_val = None
        ):
        """
        method to compute the array d = \lambda \cdot \mu if coherent_state==False
        or d = \lambda \cdot (\mu - <\mu>) if coherent_state == True
        and store to attribute self.d_array
        """

        if coherent_state == False:
            d_array = np.einsum(
                "k,ijk->ij", lambda_vector, mu_array[:n_el, :n_el, :]
            )

        else:
            _I = np.eye(n_el)
            d_array = np.einsum(
                "k,ijk->ij", lambda_vector, mu_array[:n_el, :n_el, :]
            )

            if coherent_state_val == None and coherent_state_pos == None:
                _d_exp = d_array[0, 0]
            elif coherent_state_val == None and coherent_state_pos != None:
                _d_exp = d_array[coherent_state_pos, coherent_state_pos]
            else:
                _d_exp = coherent_state_val

            d_array = d_array - _I * _d_exp

        return d_array

    def compute_energy_corrections(self, order, n_el, n_ph, omega, lambda_vector,  coherent_state = False, coherent_state_val= None, coherent_state_pos = None, ):

        n_max = n_el * n_ph
        n = 0

        # Identity matrices for each subsystem
        I_matter = np.eye(n_el)
        I_photon = np.eye(n_ph)

        # Build dipole moment matrix _d
        d_array = self.build_d_array(
                n_el, lambda_vector, self.dipoles, coherent_state, coherent_state_pos, coherent_state_val
            )
        _d = np.copy(d_array)


        # Create bosonic subspace operators
        b = self.create_annihilation_operator(n_ph)
        b_dagger = self.create_creation_operator(n_ph)
        N = self.create_number_operator(n_ph)


        # Electronic energy contribution (diagonal in matter space, identity in photon space)
        E_matter = np.diag(self.E_array[:n_el])
        E = np.kron(I_photon, E_matter)

        # Photon energy contribution (diagonal in photon space, identity in matter space)
        Cav_photon = omega * N
        Cav = np.kron( Cav_photon, I_matter)

        #Bilinear light-matter coupling term

        BLC_matter = _d
        BLC_photon = (b_dagger + b)

        BLC = - np.sqrt(omega / 2)  *  np.kron( BLC_photon, I_matter) @ np.kron(I_photon, BLC_matter)


        # Dipole -energy term
        DSE_matter = 1 / 2 * _d @ _d

        DSE = np.kron( I_photon, DSE_matter)

        # Total Hamiltonian
        H0 = E + Cav  
        V= BLC +DSE



        # Diagonalize the unperturbed Hamiltonian
        E0, psi0 = np.linalg.eigh(H0)


        #psi0[:, 0] is firt wavefunction
        print(psi0.shape)

        # Initialize corrections to energy and wavefunction
        energy_corrections = np.zeros(( order+1))
        blc_corrections = np.zeros(( order+1))
        dse_corrections = np.zeros(( order+1))
        wavefunction_corrections = np.zeros((n_max, order+1))



        # Set unperturbed energies and wavefunctions
        energy_corrections[ 0] = E0[n]
        wavefunction_corrections[ :, 0] = psi0[:, n]


        n_0 = 0


        for k in range(1, order + 1):
            
            if k == 1:
                #first order energy correction
                energy_corrections[1] = np.dot(psi0[:, n].T, np.dot(V, psi0[:,n]))
                blc_corrections[1] = np.dot(psi0[:, n].T, np.dot(BLC, psi0[:,n]))
                dse_corrections[1] = np.dot(psi0[:, n].T, np.dot(DSE, psi0[:,n]))

                print(F' 1st Order Energy Correction {energy_corrections[1]}')
                print(F' 1st Order BLC Correction    {blc_corrections[1]}')
                print(F' 1st Order DSE Correction    {dse_corrections[1]}')


                coeff_m = 0
                for m in range(0, n_max):
                    coeff_m = 0
                    if m!= n:


                        coeff_m -=   np.dot(psi0[:, m].T, np.dot(V, psi0[:,n]))

                        for l in range(1, k+1):
                            #goes from l to k-1
                            coeff_m += energy_corrections[l] * np.dot(psi0[:, m].T, wavefunction_corrections[:, k-l])

                        coeff_m = coeff_m/np.abs(E0[m] - E0[n])

                        wavefunction_corrections[:, k ] += (coeff_m * psi0[:,m])


            if k!=1:  
                for m in range(0, n_max):
                    coeff_m = 0
                    if m!= n:
                        coeff_m -=  np.dot(psi0[:,m].T, np.dot(V, wavefunction_corrections[:, k-1]))
                        for l in range(1, k+1):
                            #goes from l to k-1
                            coeff_m += energy_corrections[l] * np.dot(psi0[:, m].T, wavefunction_corrections[:, k-l])

                        coeff_m = coeff_m/np.abs(E0[m] - E0[n])
                        # print("coeff_ m for state ", m , ": ", coeff_m)

                        wavefunction_corrections[:, k ] += (coeff_m * psi0[:,m])



            if k!= 1:
                energy_correction =  np.dot(psi0[:,n].T, np.dot(V, wavefunction_corrections[:, k-1]))
                blc_correction =  np.dot(psi0[:,n].T, np.dot(BLC, wavefunction_corrections[:, k-1]))
                dse_correction =  np.dot(psi0[:,n].T, np.dot(DSE, wavefunction_corrections[:, k-1]))
                
                for j in range(0, k):
                    if j != k:
                        #print("energy correction for ", j ," : " ,energy_correction)
                        energy_correction -=  energy_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])
                        blc_correction -=  blc_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])
                        dse_correction -=  dse_corrections[j]  * np.dot(psi0[:,n].T,  wavefunction_corrections[:, k-j])

                energy_corrections[ k] = energy_correction
                blc_corrections[ k] = blc_correction
                dse_corrections[ k] = dse_correction


        #print("energy_corrections: " , energy_corrections)
        return energy_corrections, blc_corrections, dse_corrections

                    

    def PQED_Hamiltonian(self, n_el, n_ph, omega, lambda_vector,  coherent_state = False, coherent_state_val= None, coherent_state_pos = None, dipole_self_energy = True ):
        """
        Build the PF Hamiltonian for a system with n_el electronic states and n_ph photon states.
        """

        # Identity matrices for each subsystem
        I_matter = np.eye(n_el)
        I_photon = np.eye(n_ph)

        # Build dipole moment matrix _d
        d_array = self.build_d_array(
                n_el, lambda_vector, self.dipoles, coherent_state, coherent_state_pos, coherent_state_val
            )
        _d = np.copy(d_array)


        # Create bosonic subspace operators
        b = self.create_annihilation_operator(n_ph)
        b_dagger = self.create_creation_operator(n_ph)
        N = self.create_number_operator(n_ph)


        # Electronic energy contribution (diagonal in matter space, identity in photon space)
        E_matter = np.diag(self.E_array[:n_el])
        E = np.kron(I_photon, E_matter)

        # Photon energy contribution (diagonal in photon space, identity in matter space)
        Cav_photon = omega * N
        Cav = np.kron( Cav_photon, I_matter)

        #Bilinear light-matter coupling term

        BLC_matter = _d
        print("printing d")
        print(_d)
        BLC_photon = (b_dagger + b)

        BLC = - np.sqrt(omega / 2)  *  np.kron( BLC_photon, I_matter) @ np.kron(I_photon, BLC_matter)
        print("printing BLC")
        print(BLC)

        # Dipole -energy term
        DSE_matter = 1 / 2 * _d @ _d

        DSE = np.kron( I_photon, DSE_matter)
        print("Printing DSE")
        print(DSE_matter)

        # Total Hamiltonian without DSE
        H = E + Cav  + BLC

        if dipole_self_energy:
            H += DSE


        self.PCQED_MU = np.kron(I_photon, _d)


        # Return eigenvalues of the Hamiltonian
        eigenvalues, eigenvectors = np.linalg.eigh(H)
        return eigenvalues, eigenvectors, H



In [53]:
# read data from .npy files for formaldehyde casci(8,8) calculations
#Drive link for .npy files of LiH

# https://drive.google.com/drive/folders/1NQvqQ4KiXTmZZ3OjCM5nA2dqIs5ete9X?usp=sharing

# !!! Change this to the correct path on your computer!
#npy_folder = "/Users/proden/Code/POLARITONPERTURBATIONTHEORY/data/"
#npy_folder = "/Users/jfoley19/Code/data_repository/Mapol/LiH/PCQED/6-311G/r_scan/" 
npy_folder = "/Users/jfoley19/Code/qed-ci-/src/HHep_NPY/"

# these file names should still be good
E_monomer_npy_file = npy_folder + "HHep_fci_cc_pVDZ_Energies.npy"
Mu_monomer_npy_file = npy_folder + "HHep_fci_cc_pVDZ_Dipoles.npy"

# these file names should still be good
E_dimer_npy_file = npy_folder + "HHep_dimer_fci_cc_pVDZ_states_Energies.npy"
Mu_dimer_npy_file = npy_folder + "HHep_dimer_fci_cc_pVDZ_states_Dipoles.npy"

#HHep_fci_cc_pVDZ_states_Dipoles.npy
#HHep_fci_cc_pVDZ_states_Energies.npy
#HHep_dimer_fci_cc_pVDZ_states_Dipoles.npy
#HHep_dimer_fci_cc_pVDZ_states_Energies.npy

# store energy eigenvalues in E_array
E_monomer_array = np.load(E_monomer_npy_file)
# store dipole matrix elements in Mu_array
Mu_monomer_array = np.load(Mu_monomer_npy_file)

# store energy eigenvalues in E_array
E_dimer_array = np.load(E_dimer_npy_file)
# store dipole matrix elements in Mu_array
Mu_dimer_array = np.load(Mu_dimer_npy_file)

print(np.shape(E_monomer_array))
print(E_monomer_array[:5] * 2)
print(E_dimer_array[:50])


# sort these so that we know the S0, S1 state for the monomer and S0,S0; S0,S1; S1,S0; S1,S1 statesfor the dimer
E_dimer = -2.9603712197949257 * 2
print(E_monomer_array[0] * 2 -E_dimer_array[0])

(50,)
[-5.92074244 -4.29386585 -3.92849853 -3.28008121 -3.14598781]
[-5.92074239 -5.10730409 -5.10730409 -4.92462043 -4.92462043 -4.60041177
 -4.60041177 -4.54752063 -4.54752063 -4.54752063 -4.54752063 -4.53336507
 -4.53336507 -4.2938658  -4.2938658  -4.2938658  -4.1339541  -4.1339541
 -4.1339541  -4.1339541  -4.11118214 -4.11118214 -4.08019347 -4.08019347
 -4.08019347 -4.08019347 -4.0468239  -4.0468239  -3.95479844 -3.95479844
 -3.95479844 -3.95479844 -3.92849847 -3.82411023 -3.82411023 -3.78697348
 -3.78697348 -3.78697348 -3.78697348 -3.78697348 -3.78697348 -3.71992678
 -3.71992678 -3.65744795 -3.65744795 -3.65744794 -3.65744794 -3.63130572
 -3.63130572 -3.60428981]
-5.298881955440038e-08


Here we will identify the state indexes from our `E_monomer_array` and `E_dimer_array` that correspond to the following basis states:

For the monomer:
$$ |S_0\rangle, |S_1\rangle $$
with associated energies 
$$ E_0, E_1 $$
where we will call

$$ \omega = E_1 - E_0. $$

For the dimer we have the states
$$ |S_0 S_0\rangle, |S_0,S_1\rangle, |S_1,S_0\rangle, |S_1,S_1\rangle $$
with associated energies

$$ E_{00}, E_{01}, E_{10}, E_{11} $$
where 
$$ E_{00} = 2 E_0, $$

$$ E_{10} = E_{01}  = E_{00} + \omega, $$
and
$$ E_{11} = E_{00} + 2\omega. $$

In [54]:
# Monomer S0 -> E_monomer_array[0]
# Monomer S0 -> E_monomer_array[2]
E_S0 = E_monomer_array[0]
E_S1 = E_monomer_array[2]

omega = E_monomer_array[2]-E_monomer_array[0]

# Dimer S0S0 -> E_dimer_array[0]
E_S0S0 = E_dimer_array[0]

# Dimer S0S1 -> E_dimer_array[3]
E_S0S1 = E_dimer_array[3]

# Dimer S1S0 -> E_dimer_array[4]
E_S1S0 = E_dimer_array[4]

# Dimer S1S1 -> E_dimer_array[32]

S0_idx = 0
S1_idx = 2
S0S0_idx = 0
S0S1_idx = 3
S1S0_idx = 4
S1S1_idx = 32

E_S0S0_p_Om = E_S0S0 + omega
E_S0S0_p_tom = E_S0S0 + 2 * omega

print(F'Omega is         {omega}')
print(F'E_S0 energy is   {E_S0}')
print(F'E_S1 energy is   {E_S1}')
print(F'E_S0S0 energy is {E_S0S0}')
print(F'E_S0S0 + w    is {E_S0S0_p_Om}')
print(F'E_S0S1 energy is {E_S0S1}')
print(F'E_S1S0 energy is {E_S1S0}')
print(F'E_S0S0 + 2w   is {E_S0S0_p_tom}')
print(F'E_S1S1        is {E_dimer_array[32]}')

Omega is         0.996121956243931
E_S0 energy is   -2.9603712197949257
E_S1 energy is   -1.9642492635509947
E_S0S0 energy is -5.920742386601032
E_S0S0 + w    is -4.924620430357101
E_S0S1 energy is -4.924620431017138
E_S1S0 energy is -4.924620430001991
E_S0S0 + 2w   is -3.92849847411317
E_S1S1        is -3.9284984744181197


Now that we have found the indices for our basis states, we will assemble the dipole matrix in this basis such
that we have matrix elements like the following for the monomer:

\begin{equation}
\mu_{monomer} = 
\begin{bmatrix}
\langle S_0 | \hat{\mu} | S_0\rangle & \langle S_0 | \hat{\mu} | S_1\rangle \\
\langle S_1 | \hat{\mu} | S_0\rangle& \langle S_1 | \hat{\mu} | S_1\rangle
\end{bmatrix}
\end{equation}
and for the dimer
\begin{equation}
\mu_{dimer} = 
\begin{bmatrix}
\langle S_0,S_0 | \hat{\mu} | S_0,S_0\rangle & \langle S_0,S_0 | \hat{\mu} | S_0,S_1\rangle & \langle S_0,S_0 | \hat{\mu} | S_1,S_0\rangle & \langle S_0,S_0 | \hat{\mu} | S_1,S_1\rangle \\
\langle S_0,S_1 | \hat{\mu} | S_0,S_0\rangle & \langle S_0,S_1 | \hat{\mu} | S_0,S_1\rangle & \langle S_0,S_1 | \hat{\mu} | S_1,S_0\rangle & \langle S_0,S_1 | \hat{\mu} | S_1,S_1\rangle \\
\langle S_1,S_0 | \hat{\mu} | S_0,S_0\rangle & \langle S_1,S_0 | \hat{\mu} | S_0,S_1\rangle & \langle S_1,S_0 | \hat{\mu} | S_1,S_0\rangle & \langle S_1,S_0 | \hat{\mu} | S_1,S_1\rangle \\
\langle S_1,S_1 | \hat{\mu} | S_0,S_0\rangle & \langle S_1,S_1 | \hat{\mu} | S_0,S_1\rangle & \langle S_1,S_1 | \hat{\mu} | S_1,S_0\rangle & \langle S_1,S_1 | \hat{\mu} | S_1,S_1\rangle 
\end{bmatrix}
\end{equation}
 

In [55]:
# initialize matrices
# dipole for monomer
mu_monomer = np.zeros((2,2,3))

# dipole for monomer but only including transition dipole moments
mu_monomer_tdm_only = np.zeros((2,2,3))

# dipole for dimer 
mu_dimer = np.zeros((4,4,3))

# dipole for dimer but only including transition dipole moments
mu_dimer_tdm_only = np.zeros((4,4,3))

e_monomer = np.zeros(2)

e_dimer = np.zeros(4)



# build monomer matrix
e_monomer[0] = E_monomer_array[S0_idx]
e_monomer[1] = E_monomer_array[S1_idx]

e_dimer[0] = E_dimer_array[S0S0_idx]
e_dimer[1] = E_dimer_array[S0S1_idx]
e_dimer[2] = E_dimer_array[S1S0_idx]
e_dimer[3] = E_dimer_array[S1S1_idx]

# tdms
mu_monomer[0,1,:] = Mu_monomer_array[S0_idx, S1_idx, :]
mu_monomer[1,0,:] = Mu_monomer_array[S1_idx, S0_idx, :]
#mu_monomer[1,1,:] = Mu_monomer_array[S1_idx, S1_idx, :] 

# get tdm only version
mu_monomer_tdm_only = np.copy(mu_monomer)

# get static dipoles
mu_monomer[0,0,:] = Mu_monomer_array[S0_idx, S0_idx, :] 
mu_monomer[1,1,:] = Mu_monomer_array[S1_idx, S1_idx, :] 

print("Monomer Mu")
print(mu_monomer)

print("Monomer Mu TDM Only")
print(mu_monomer_tdm_only)

# build dimer matrix

# standard tdms only 
mu_dimer[0,1,:] = Mu_dimer_array[S0S0_idx, S0S1_idx, : ]
mu_dimer[0,2,:] = Mu_dimer_array[S0S0_idx, S1S0_idx, : ]
mu_dimer[1,0,:] = Mu_dimer_array[S0S1_idx, S0S0_idx, : ]
mu_dimer[2,0,:] = Mu_dimer_array[S1S0_idx, S0S0_idx, : ]


# get tdm only version
mu_dimer_tdm_only = np.copy(mu_dimer)

# get non-standard tdms and static dipoles
mu_dimer[0,0,:] = Mu_dimer_array[S0S0_idx, S0S0_idx, : ]
mu_dimer[0,3,:] = Mu_dimer_array[S0S0_idx, S1S1_idx, : ]
mu_dimer[1,1,:] = Mu_dimer_array[S0S1_idx, S0S1_idx, : ]
mu_dimer[2,2,:] = Mu_dimer_array[S1S0_idx, S1S0_idx, : ]
mu_dimer[1,2,:] = Mu_dimer_array[S0S1_idx, S1S0_idx, : ]
mu_dimer[1,3,:] = Mu_dimer_array[S0S1_idx, S1S1_idx, : ]

mu_dimer[2,1,:] = Mu_dimer_array[S1S0_idx, S0S1_idx, : ]
mu_dimer[2,3,:] = Mu_dimer_array[S1S0_idx, S1S1_idx, : ]
mu_dimer[3,0,:] = Mu_dimer_array[S1S1_idx, S0S0_idx, : ]
mu_dimer[3,1,:] = Mu_dimer_array[S1S1_idx, S0S1_idx, : ]
mu_dimer[3,2,:] = Mu_dimer_array[S1S1_idx, S1S0_idx, : ]
mu_dimer[3,3,:] = Mu_dimer_array[S1S1_idx, S1S1_idx, : ]

print("Dimer Mu")
print(mu_dimer)

print("Dimer Mu TDM Only")
print(mu_dimer_tdm_only)


Monomer Mu
[[[ 8.83020351e-17 -2.78314221e-17 -6.43777131e-01]
  [ 4.53112073e-17  1.30485611e-16 -7.60294804e-01]]

 [[ 4.53112073e-17  1.30485611e-16 -7.60294804e-01]
  [-1.94315467e-16 -1.00440872e-16  4.20099337e-01]]]
Monomer Mu TDM Only
[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
  [ 4.53112073e-17  1.30485611e-16 -7.60294804e-01]]

 [[ 4.53112073e-17  1.30485611e-16 -7.60294804e-01]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]]
Dimer Mu
[[[ 1.62926843e-13 -2.60397662e-13 -1.28755433e+00]
  [ 2.04797901e-13  5.96950697e-14 -7.60290813e-01]
  [ 3.16541323e-13  8.57706712e-14 -7.60298803e-01]
  [-1.01641990e-12 -4.33750726e-13  3.89773787e-12]]

 [[ 2.04797901e-13  5.96950697e-14 -7.60290815e-01]
  [-3.28235081e-13  2.42204778e-13 -2.23677861e-01]
  [ 4.13606217e-13 -1.11475535e-13  1.91846539e-12]
  [ 4.66962077e-13  2.47158830e-13  7.60290807e-01]]

 [[ 3.16541323e-13  8.57706712e-14 -7.60298803e-01]
  [ 4.13606217e-13 -1.11475535e-13  1.91846539e-12]
  [ 1.444175

Create an instance of the PT_arbitrary class for monomer using the full dipole matrix and the monomer only using TDMs

In [56]:
monomer_inst = PT_arbitrary(e_monomer, mu_monomer)
monomer_tdm_only_inst = PT_arbitrary(e_monomer, mu_monomer_tdm_only)

Build Rabi Hamiltonian's using the full dipole matrix and only the TDMs and get the eigenvalues and Rabi splitting

In [57]:
# define lambda vector
lambda_vector = np.array([0, 0, 0.05])
# define number of photonic Fock states
N_ph = 2

# get spectrum of system with full dipole matrix (Rabi Hamiltonian, no dipole self energy)
full_monomer_eigs, full_monomer_vecs, full_monomer_H  = monomer_inst.PQED_Hamiltonian(2, N_ph, omega, lambda_vector, dipole_self_energy=False)

# get spectrum of system with tdm-only matrix (Rabi Hamiltonian, no dipole self energy)
tdm_monomer_eigs, tdm_monomer_vecs, tdm_monomer_H  = monomer_tdm_only_inst.PQED_Hamiltonian(2, N_ph, omega, lambda_vector, dipole_self_energy=False)

# print Rabi Hamiltonian with the full dipole matrix
print("Full H")
print(full_monomer_H - E_S0 * np.eye(4))

# print Rabi Hamiltonian with the tdm-only dipole matrix
print("TDM Only H")
print(tdm_monomer_H - E_S0 * np.eye(4))
print(E_S0 * np.eye(4))

printing d
[[-0.03218886 -0.03801474]
 [-0.03801474  0.02100497]]
printing BLC
[[ 0.          0.          0.02271678  0.02682831]
 [ 0.          0.          0.02682831 -0.01482393]
 [ 0.02271678  0.02682831  0.          0.        ]
 [ 0.02682831 -0.01482393  0.          0.        ]]
Printing DSE
[[0.00124062 0.00021258]
 [0.00021258 0.00094316]]
printing d
[[ 0.         -0.03801474]
 [-0.03801474  0.        ]]
printing BLC
[[0.         0.         0.         0.02682831]
 [0.         0.         0.02682831 0.        ]
 [0.         0.02682831 0.         0.        ]
 [0.02682831 0.         0.         0.        ]]
Printing DSE
[[0.00072256 0.        ]
 [0.         0.00072256]]
Full H
[[ 0.          0.          0.02271678  0.02682831]
 [ 0.          0.99612196  0.02682831 -0.01482393]
 [ 0.02271678  0.02682831  0.99612196  0.        ]
 [ 0.02682831 -0.01482393  0.          1.99224391]]
TDM Only H
[[0.         0.         0.         0.02682831]
 [0.         0.99612196 0.02682831 0.        ]
 [0

In [58]:
rabi_splitting_full_monomer = full_monomer_eigs[2]-full_monomer_eigs[1]
print(F" Full Monomer Rabi splitting is {rabi_splitting_full_monomer}")

rabi_splitting_tdm_monomer = tdm_monomer_eigs[2]-tdm_monomer_eigs[1]
print(F" TDM Monomer Rabi splitting is {rabi_splitting_tdm_monomer}")

 Full Monomer Rabi splitting is 0.05366001545791943
 TDM Monomer Rabi splitting is 0.05365661624221585


In [59]:
# define lambda vector
lambda_vector = np.array([0, 0, 0.05])
# define number of photonic Fock states
N_ph = 2

# get spectrum of system with full dipole matrix (Rabi Hamiltonian, no dipole self energy)
full_PF_monomer_eigs, full_PF_monomer_vecs, full_PF_monomer_H  = monomer_inst.PQED_Hamiltonian(2, N_ph, omega, lambda_vector, dipole_self_energy=True)

# get spectrum of system with tdm-only matrix (Rabi Hamiltonian, no dipole self energy)
tdm_PF_monomer_eigs, tdm_PF_monomer_vecs, tdm_PF_monomer_H  = monomer_tdm_only_inst.PQED_Hamiltonian(2, N_ph, omega, lambda_vector, dipole_self_energy=True)

# print Rabi Hamiltonian with the full dipole matrix
print("PF H")
print(full_PF_monomer_H - E_S0 * np.eye(4))

# print Rabi Hamiltonian with the tdm-only dipole matrix
print("PF TDM H")
print(tdm_PF_monomer_H - E_S0 * np.eye(4))


printing d
[[-0.03218886 -0.03801474]
 [-0.03801474  0.02100497]]
printing BLC
[[ 0.          0.          0.02271678  0.02682831]
 [ 0.          0.          0.02682831 -0.01482393]
 [ 0.02271678  0.02682831  0.          0.        ]
 [ 0.02682831 -0.01482393  0.          0.        ]]
Printing DSE
[[0.00124062 0.00021258]
 [0.00021258 0.00094316]]
printing d
[[ 0.         -0.03801474]
 [-0.03801474  0.        ]]
printing BLC
[[0.         0.         0.         0.02682831]
 [0.         0.         0.02682831 0.        ]
 [0.         0.02682831 0.         0.        ]
 [0.02682831 0.         0.         0.        ]]
Printing DSE
[[0.00072256 0.        ]
 [0.         0.00072256]]
PF H
[[ 1.24062148e-03  2.12576331e-04  2.27167819e-02  2.68283081e-02]
 [ 2.12576331e-04  9.97065121e-01  2.68283081e-02 -1.48239267e-02]
 [ 2.27167819e-02  2.68283081e-02  9.97362578e-01  2.12576331e-04]
 [ 2.68283081e-02 -1.48239267e-02  2.12576331e-04  1.99318708e+00]]
PF TDM H
[[7.22560237e-04 0.00000000e+00 0.000

In [60]:
rabi_splitting_full_PF_monomer = full_PF_monomer_eigs[2]-full_PF_monomer_eigs[1]
print(F" Full PF Monomer Rabi splitting is {rabi_splitting_full_PF_monomer}")

rabi_splitting_tdm_PF_monomer = tdm_PF_monomer_eigs[2]-tdm_PF_monomer_eigs[1]
print(F" TDM PF Monomer Rabi splitting is {rabi_splitting_tdm_PF_monomer}")

 Full PF Monomer Rabi splitting is 0.05368093280283048
 TDM PF Monomer Rabi splitting is 0.05365661624221452


Instantiate for the dimer 

In [61]:
dimer_inst = PT_arbitrary(e_dimer, mu_dimer)
dimer_tdm_only_inst = PT_arbitrary(e_dimer, mu_dimer_tdm_only)

In [62]:
# get spectrum of system with full dipole matrix (Rabi Hamiltonian, no dipole self energy)
full_PF_dimer_eigs, full_PF_dimer_vecs, full_PF_dimer_H  = dimer_inst.PQED_Hamiltonian(4, N_ph, omega, lambda_vector, dipole_self_energy=True)

# get spectrum of system with tdm-only matrix (Rabi Hamiltonian, no dipole self energy)
tdm_PF_dimer_eigs, tdm_PF_dimer_vecs, tdm_PF_dimer_H  = dimer_tdm_only_inst.PQED_Hamiltonian(4, N_ph, omega, lambda_vector, dipole_self_energy=True)

# print Rabi Hamiltonian with the full dipole matrix
print("PF H")
print(full_PF_dimer_H - E_S0S0 * np.eye(8))

# print Rabi Hamiltonian with the tdm-only dipole matrix
print("PF TDM H")
print(tdm_PF_dimer_H - E_S0S0 * np.eye(8))

printing d
[[-6.43777166e-02 -3.80145407e-02 -3.80149402e-02  1.94886894e-13]
 [-3.80145408e-02 -1.11838930e-02  9.59232693e-14  3.80145404e-02]
 [-3.80149401e-02  9.59232693e-14 -1.11838914e-02  3.80149405e-02]
 [ 1.94886894e-13  3.80145404e-02  3.80149406e-02  4.20099315e-02]]
printing BLC
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.54335662e-02  2.68281673e-02  2.68284492e-02 -1.37538376e-13]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.68281674e-02  7.89285752e-03 -6.76963465e-14 -2.68281671e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   2.68284492e-02 -6.76963465e-14  7.89285640e-03 -2.68284495e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.37538376e-13 -2.68281671e-02 -2.68284495e-02 -2.96478519e-02]
 [ 4.54335662e-02  2.68281673e-02  2.68284492e-02 -1.37538376e-13
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.68281674e-02  7.89285752e-03 -6.76963